In [1]:
import pandas as pd

In [2]:
schoolsdata = "Generators/PyCitySchools/generated_data/schools_complete.csv"
studentsdata = "Generators/PyCitySchools/generated_data/students_complete.csv"

In [3]:
schooldata_df = pd.read_csv(schoolsdata)

In [4]:
studentdata_df = pd.read_csv(studentsdata)

In [5]:
#DISTRICT SUMMARY
#Total Schools
totalschools = schooldata_df["school_name"].count()
#Total Students
totalstudents = studentdata_df["student_name"].count()
#Total Budget
totalbudget = schooldata_df["budget"].sum()
#Average Math Score
avgmath = studentdata_df["math_score"].mean()
#Average Reading Score
avgreading = studentdata_df["reading_score"].mean()
#% Passing Math
passmath = studentdata_df[(studentdata_df["math_score"] > 70)].count()["math_score"]
failmath = studentdata_df[(studentdata_df["math_score"] <= 70)].count()["math_score"]
percpassmath = (passmath/(passmath + failmath))*100
#% Passing Reading
passreading = studentdata_df[(studentdata_df["reading_score"] > 70)].count()["reading_score"]
failreading = studentdata_df[(studentdata_df["reading_score"] <= 70)].count()["reading_score"]
percpassreading = (passreading/(passreading + failreading))*100
#Overall Passing Rate (Average of the above two)
overallpass = (percpassmath + percpassreading)/2

In [6]:
#Order for District Summary
districtsummary = pd.DataFrame({"Total School": [totalschools],
                                   "Total Students": [totalstudents],
                                   "Total Budget": [totalbudget],
                                   "Average Math Score": [avgmath],
                                   "Average Reading Score": [avgreading],
                                   "% Passing Math":[percpassmath],
                                   "% Passing Reading":[percpassreading],
                                   "% Overall Passing Rate": [overallpass]})

In [7]:
#Formatting for District Summary
districtsummary["Total Students"] = districtsummary["Total Students"].map("{0:,.0f}".format)
districtsummary["Total Budget"] = districtsummary["Total Budget"].map("${0:,.0f}".format)
districtsummary["% Passing Math"] = districtsummary["% Passing Math"].map("{0:,.2f}%".format)
districtsummary["% Passing Reading"] = districtsummary["% Passing Reading"].map("{0:,.2f}%".format)
districtsummary["% Overall Passing Rate"] = districtsummary["% Overall Passing Rate"].map("{0:,.2f}%".format)

ds = districtsummary[["Total School","Total Students","Total Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading", "% Overall Passing Rate"]]
ds

,Total School,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,11,"29,376","$18,648,468",82.269846,82.865877,84.12%,76.81%,80.47%


In [8]:
#dataframe with passing math score grouped by school name
math_df = studentdata_df.loc[studentdata_df["math_score"] > 70]
mathgroup_df = math_df.groupby(["school_name"]).count()
mathgroup_df.pop("Student ID")
mathgroup_df.pop("student_name")
mathgroup_df.pop("gender")
mathgroup_df.pop("grade")
mathgroup_df.pop("reading_score")
mathgroup_df.reset_index(inplace=True)

In [9]:
#dataframe with passing reading score grouped by school name
read_df = studentdata_df.loc[studentdata_df["reading_score"] > 70]
readgroup_df = read_df.groupby(["school_name"]).count()
readgroup_df.pop("Student ID")
readgroup_df.pop("student_name")
readgroup_df.pop("gender")
readgroup_df.pop("grade")
readgroup_df.pop("math_score")
readgroup_df.reset_index(inplace=True)

In [10]:
#Merge reading and math scores df
mathread = pd.merge(mathgroup_df, readgroup_df, left_on="school_name", right_on="school_name")

In [11]:
#total students df
totstu = studentdata_df.loc[studentdata_df["Student ID"]]
totalstudentdf = totstu.groupby(["school_name"]).count()
totalstudentdf.pop("reading_score")
totalstudentdf.pop("student_name")
totalstudentdf.pop("gender")
totalstudentdf.pop("grade")
totalstudentdf.pop("math_score")
totalstudentdf.reset_index(inplace=True)

In [12]:
#Calculate %math,reading,overall
tocalcperc = pd.merge(mathread, totalstudentdf, left_on="school_name", right_on="school_name")

percmath = (tocalcperc["math_score"] / tocalcperc["Student ID"])*100
tocalcperc["% Passing Math"] = percmath
percread = (tocalcperc["reading_score"] / tocalcperc["Student ID"])*100
tocalcperc["% Passing Reading"] = percread
percoverall = (percmath+percread)/2
tocalcperc["% Passing Overall"] = percoverall

perscentpassing = tocalcperc

In [13]:
#only have %math,reading,overall
perscentpassing.pop("math_score")
perscentpassing.pop("reading_score")
perscentpassing.pop("Student ID");

In [14]:
#avg reading score and avg math score
student2 = studentdata_df.groupby(["school_name"])
studentmean= student2.mean()
studentmean.pop("Student ID")
studentmean = studentmean.rename(columns={"reading_score":"Average Reading Score", "math_score":"Average Math Score"})
studentmean.reset_index(inplace=True)

In [15]:
#total students, total school budget, per student budget
school2 = schooldata_df.groupby(["school_name","type"])
schoolsum = school2.mean()
schoolsum.pop("School ID")
schoolsum = schoolsum.rename(columns={"size":"Total Students", "budget":"Total School Budget"})
perstubud = schoolsum["Total School Budget"] / schoolsum["Total Students"]
schoolsum["Per Student Budget"] = perstubud
schoolsum.reset_index(inplace=True)

In [16]:
#Merge perscentpassing, studentmean, schoolsum
perscentstudent = pd.merge(perscentpassing, studentmean, left_on="school_name", right_on="school_name")
perscentstudentschool = pd.merge(perscentstudent, schoolsum, left_on="school_name", right_on="school_name")
perscentstudentschool.set_index('school_name', inplace=True)
perscentstudentschool.index.name = None
perscentstudentschool
forscoresby = perscentstudentschool
forscoresby1 = perscentstudentschool.copy(deep=True)
forscoresby2 = perscentstudentschool.copy(deep=True)

In [17]:
#Format & Order for School Summary
perscentstudentschool = perscentstudentschool.rename(columns={"type":"School Type"})

formatss = perscentstudentschool
formatss["Total School Budget"] = formatss["Total School Budget"].map("${:,.2f}".format)
formatss["Per Student Budget"] = formatss["Per Student Budget"].map("${:,.2f}".format)


schoolsummary = formatss[["School Type","Total Students","Total School Budget", "Per Student Budget", 
                      "Average Math Score","Average Reading Score","% Passing Math","% Passing Reading", "% Passing Overall"]]
schoolsummary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Campbell High School,Charter,271,"$157,993.00",$583.00,83.594096,93.771218,92.619926,100.000000,96.309963
Galloway High School,Charter,2471,"$1,445,535.00",$585.00,83.566168,94.029543,90.813436,100.000000,95.406718
Glass High School,District,3271,"$2,155,589.00",$659.00,81.293183,76.888108,79.333537,65.117701,72.225619
Gomez High School,Charter,2154,"$1,288,092.00",$598.00,83.838440,94.027391,90.807799,100.000000,95.403900
Gonzalez High School,Charter,1855,"$1,192,765.00",$643.00,83.442588,94.140701,89.649596,100.000000,94.824798
Hawkins High School,District,4555,"$2,851,430.00",$626.00,81.723820,77.005928,81.668496,64.983535,73.326015
Kelly High School,District,3307,"$2,225,611.00",$673.00,81.678258,76.829755,80.586634,64.166919,72.376777
Macdonald High School,Charter,901,"$550,511.00",$611.00,83.779134,93.932297,92.230855,100.000000,96.115427
Miller High School,Charter,2424,"$1,418,040.00",$585.00,83.610149,93.997525,90.800330,100.000000,95.400165
Sherman High School,District,3213,"$2,152,710.00",$670.00,81.502023,77.290694,80.547775,65.141612,72.844693


In [18]:
#Top Performing Schools (By Passing Rate)
topschools = schoolsummary.sort_values("% Passing Overall", ascending=False)
topschools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Campbell High School,Charter,271,"$157,993.00",$583.00,83.594096,93.771218,92.619926,100.0,96.309963
Macdonald High School,Charter,901,"$550,511.00",$611.00,83.779134,93.932297,92.230855,100.0,96.115427
Galloway High School,Charter,2471,"$1,445,535.00",$585.00,83.566168,94.029543,90.813436,100.0,95.406718
Gomez High School,Charter,2154,"$1,288,092.00",$598.00,83.838440,94.027391,90.807799,100.0,95.403900
Miller High School,Charter,2424,"$1,418,040.00",$585.00,83.610149,93.997525,90.800330,100.0,95.400165


In [19]:
#Bottom Performing Schools (By Passing Rate)
bottomschools = schoolsummary.sort_values("% Passing Overall")
bottomschools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Glass High School,District,3271,"$2,155,589.00",$659.00,81.293183,76.888108,79.333537,65.117701,72.225619
Kelly High School,District,3307,"$2,225,611.00",$673.00,81.678258,76.829755,80.586634,64.166919,72.376777
Smith High School,District,4954,"$3,210,192.00",$648.00,81.539160,77.146952,80.682277,64.251110,72.466694
Sherman High School,District,3213,"$2,152,710.00",$670.00,81.502023,77.290694,80.547775,65.141612,72.844693
Hawkins High School,District,4555,"$2,851,430.00",$626.00,81.723820,77.005928,81.668496,64.983535,73.326015


In [20]:
#MATH SCORES BY GRADE
mathbygrade = studentdata_df.groupby(["school_name","grade"]).mean()
mathbygrade.pop("Student ID")
mathbygrade.pop("reading_score")

mathbygrade1 = mathbygrade.groupby(["school_name","grade"])["math_score"].mean().unstack()
mathbygrade1= mathbygrade1[["9th","10th","11th","12th"]]
mathbygrade1.index.name = None
mathbygrade1 = mathbygrade1.rename_axis("", axis =1)
mathbygrade1

,9th,10th,11th,12th
Campbell High School,83.842857,84.269663,83.940000,82.064516
Galloway High School,83.534384,83.551630,83.975425,83.204724
Glass High School,81.867647,81.044652,81.390935,80.823120
Gomez High School,83.676568,83.966817,83.874468,83.828916
Gonzalez High School,83.548263,83.952118,83.201970,82.840206
Hawkins High School,81.667758,81.475371,81.885770,81.938296
Kelly High School,81.789659,81.881168,81.497283,81.453920
Macdonald High School,84.255507,83.813953,83.482906,83.516484
Miller High School,83.823713,83.624661,83.635838,83.304183
Sherman High School,81.496614,81.526882,81.232117,81.735955


In [21]:
#READING SCORES BY GRADE
readbygrade = studentdata_df.groupby(["school_name","grade"]).mean()
readbygrade.pop("Student ID")
readbygrade.pop("math_score")

readbygrade1 = readbygrade.groupby(["school_name","grade"])["reading_score"].mean().unstack()
readbygrade1= readbygrade1[["9th","10th","11th","12th"]]
readbygrade1.index.name = None
readbygrade1 = readbygrade1.rename_axis("", axis =1)
readbygrade1

,9th,10th,11th,12th
Campbell High School,93.471429,93.876404,94.080000,93.709677
Galloway High School,94.065903,93.961957,93.979206,94.129921
Glass High School,76.444570,77.319834,77.128895,76.618384
Gomez High School,94.186469,93.972851,93.808511,94.130120
Gonzalez High School,94.042471,94.103131,94.416256,94.036082
Hawkins High School,76.518003,77.174355,77.526621,76.852106
Kelly High School,76.367803,77.267875,76.637228,76.966988
Macdonald High School,94.048458,94.135659,93.799145,93.670330
Miller High School,93.897036,94.039295,94.238921,93.823194
Sherman High School,77.292325,77.111828,77.312409,77.501404


In [22]:
#Scores by School Spending
forscoresby["Per Student Budget"].astype(float)
s_spending = forscoresby
s_spending.pop("type")
s_spending.pop("Total Students")
s_spending.pop("Total School Budget")
schoolspending = s_spending
schoolspending
spending_bins = [0, 585, 615, 645, 675]
spending_group = ['<$585', '$585-615', '$615-645', '$645-675']
schoolspending["Spending Ranges (Per Student)"] = pd.cut(schoolspending["Per Student Budget"], spending_bins, labels=spending_group)
schoolspending_groups = schoolspending.groupby("Spending Ranges (Per Student)")
ssg = schoolspending_groups[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading", "% Passing Overall"]]
ssg.max()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Spending Ranges (Per Student),,,,,
<$585,83.610149,94.029543,92.619926,100.000000,96.309963
$585-615,83.838440,94.027391,92.230855,100.000000,96.115427
$615-645,83.442588,94.140701,89.649596,100.000000,94.824798
$645-675,81.678258,77.290694,80.682277,65.141612,72.844693


In [23]:
#Scores by School Size
s_size = forscoresby1
s_size.pop("type")
s_size.pop("Per Student Budget")
s_size.pop("Total School Budget")
schoolsize = s_size
schoolsize
size_bins = [0, 1000, 2000, 5000]
size_group = ['Small(<1000)', 'Medium (1000-2000)', 'Large (2000-5000)']
schoolsize["School Size"] = pd.cut(schoolsize["Total Students"], size_bins, labels=size_group)
schoolsize_groups = schoolsize.groupby("School Size")
ssizeg = schoolsize_groups[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading", "% Passing Overall"]]
ssizeg.max()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Size,,,,,
Small(<1000),83.779134,93.932297,92.619926,100.0,96.309963
Medium (1000-2000),83.442588,94.140701,89.649596,100.0,94.824798
Large (2000-5000),83.838440,94.029543,90.813436,100.0,95.406718


In [24]:
#Scores by School Type
s_type = forscoresby2
s_type.pop("Total Students")
s_type.pop("Per Student Budget")
s_type.pop("Total School Budget")
schooltype = s_type
typegroup = schooltype.groupby(["type"]).mean()
typegroup = typegroup[["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading", "% Passing Overall"]]
typegroup.index.names = ['School Type']
typegroup

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
School Type,,,,,
Charter,83.638429,93.983112,91.153657,100.000000,95.576828
District,81.547289,77.032287,80.563744,64.732175,72.647960


Three Observable Trends:
1. Schools with smaller student body size have a higher percentage of passing students.
2. Charter school students are more likely to succeed in math and reading than district school students.
3. Schools that have higher spending ranges per student are more likely to have failing reading scores compared to students in schools with lower spending ranges.
